# To do before starting the notebook

- Open a terminal in the app folder
- Start the mlflow backend:
    - mlflow server --backend-store-uri sqlite:///mlflow.db



In [ ]:
import matplotlib.pyplot as plt
from typing import Literal
import pandas as pd
import numpy as np
import gc


#Data pre processing 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#Scoring
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier

#Hyperparam opti
import optuna
from optuna.pruners import MedianPruner

#Mlflow
import mlflow
import mlflow.sklearn

# Explainability
import shap

import warnings
warnings.simplefilter(action='ignore', category=AttributeError)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def clear_global_variable(variable_name):
    """
    Check if a global variable exists and delete it if it does.
    """
    for v in variable_name:
        if v in globals():
            # Delete the global variable
            del globals()[v]
    gc.collect()


def optimize_int_memory(df:pd.DataFrame):
    # Step 1: Refactor dtypes to save space
    print(df.dtypes.value_counts())
    initial_memory_usage = df.memory_usage(deep=True).sum()
    print(f"Initial memory usage: {initial_memory_usage / 1024**2:.2f} MB")
    
    # Check each int64 column
    for col in df.select_dtypes(include=['int64','Int64']).columns:
        max_val = df[col].max()
        min_val = df[col].min()
        
        if min_val >= 0:
            if max_val <= 255:
                df[col] = df[col].astype('uint8')
            elif max_val <= 65535:
                df[col] = df[col].astype('uint16')
            elif max_val <= 4294967295:
                df[col] = df[col].astype('uint32')
        else:
            if min_val >= -128 and max_val <= 127:
                df[col] = df[col].astype('int8')
            elif min_val >= -32768 and max_val <= 32767:
                df[col] = df[col].astype('int16')
            elif min_val >= -2147483648 and max_val <= 2147483647:
                df[col] = df[col].astype('int32')
    
    
    print(df.dtypes.value_counts())
    final_memory_usage = df.memory_usage(deep=True).sum()
    print(f"Final memory usage: {final_memory_usage / 1024**2:.2f} MB")
    return df

In [ ]:
def change_dataset_version(dataset_version : Literal['1.0','1.1','2.0','3.0']):
    """
    1.0 : Full dataset from our data kernel with Na
    1.1 : Full dataset from our data kernel `without` Na (imputed by mean)
    2.0 : sampled data, by default 10% of the original data
    2.1 : sampled data, by default 10% of the original data `without` Na (imputed by mean) 
    3.0 : scaled and sampled data without Na, StandardScaler and 10%
    4.0 : scaled and PCA
    4.1 : scaled and PCA and sampled, by default 10% of the original data size 

    # How to use:
    `X, y, current_version = change_dataset_version('1.0')`
    """
    reset_global_var = ['X','y','current_version']
    clear_global_variable(reset_global_var)

    if dataset_version == '1.0':
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        train_data.drop(columns='level_0', inplace=True)

        #y is just the target column
        y = train_data['TARGET'].copy()
        y = 1 - y

        # Get the columns to be included in X
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]
        #Remove characters that model can't read
        X.columns = X.columns.str.replace('[^\w\s]','')
        for col in X.select_dtypes(include=['Int64']).columns:
            X[col] = X[col].astype('float64')
        #Clean unused data to free up ram
        del train_data
        gc.collect()
        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    elif dataset_version == '1.1':
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        train_data.drop(columns='level_0', inplace=True)

        # Replace positive infinity values
        for col in train_data.columns:
            max_val = train_data[train_data[col] != np.inf][col].max()
            train_data[col].replace(np.inf, max_val, inplace=True)
        
        # Replace negative infinity values (if applicable)
        for col in train_data.columns:
            min_val = train_data[train_data[col] != -np.inf][col].min()
            train_data[col].replace(-np.inf, min_val, inplace=True)

        # Calculate the mean of each column
        column_means = train_data.mean()
        #Identify int col:
        int_columns = train_data.select_dtypes(include=['int64', 'int32', 'int8', 'Int64']).columns
        #round the mean for those columns:
        column_means[int_columns] = column_means[int_columns].round()
        # Fill NaN values in each column with the corresponding mean value
        train_data.fillna(column_means, inplace=True)

        train_data = optimize_int_memory(train_data)
        
        #y is just the target column
        y = train_data['TARGET'].copy()
        y= 1 - y

        # Get the columns to be included in X
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]
        #Remove characters that model can't read
        X.columns = X.columns.str.replace('[^\w\s]','')
        #Clean unused data to free up ram
        del train_data
        gc.collect()
        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    elif dataset_version == '2.0':
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        train_data.drop(columns='level_0', inplace=True)

        # Sample 10% of the data
        train_data = train_data.sample(frac=0.10, random_state=42)
        train_data.reset_index(inplace=True)
        #Remove characters that model can't read
        train_data.columns = train_data.columns.str.replace('[^\w\s]','')
        #Save the df for dashboard purposes
        #train_data.to_csv('./Dashboard/sampled_train_data.csv')
        
        #y is just the target column
        y = train_data['TARGET'].copy().astype(int)
        y = 1 - y
        # Get the columns to be included in X
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]
        for col in X.select_dtypes(include=['Int64']).columns:
            X[col] = X[col].astype('float64')
        #Clean unused data to free up ram
        del train_data
        gc.collect()

        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    elif dataset_version == '2.1':
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        train_data.drop(columns='level_0', inplace=True)

        # Sample 10% of the data
        train_data = train_data.sample(frac=0.10, random_state=42)


        # Replace positive infinity values
        for col in train_data.columns:
            max_val = train_data[train_data[col] != np.inf][col].max()
            train_data[col].replace(np.inf, max_val, inplace=True)

        # Replace negative infinity values
        for col in train_data.columns:
            min_val = train_data[train_data[col] != -np.inf][col].min()
            train_data[col].replace(-np.inf, min_val, inplace=True)

        # Calculate the mean of each column
        column_means = train_data.mean()
        #Identify int col:
        int_columns = train_data.select_dtypes(include=['int64', 'int32', 'int8', 'Int64']).columns
        #round the mean for those columns:
        column_means[int_columns] = column_means[int_columns].round()
        # Fill NaN values in each column with the corresponding mean value
        train_data.fillna(column_means, inplace=True)

        #Restructure the dtypes for memory optimisation
        train_data = optimize_int_memory(train_data)
        #y is just the target column
        y = train_data['TARGET'].copy()
        y = 1 - y

        # Get the columns to be included in X
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]
        #Remove characters that model can't read
        X.columns = X.columns.str.replace('[^\w\s]','')

        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
        #Clean unused data to free up ram
        del train_data
        gc.collect()


    elif dataset_version == '3.0':
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        train_data.drop(columns='level_0', inplace=True)

        # Sample 10% of the data
        train_data = train_data.sample(frac=0.10, random_state=42)


        # Replace positive infinity values
        for col in train_data.columns:
            max_val = train_data[train_data[col] != np.inf][col].max()
            train_data[col].replace(np.inf, max_val, inplace=True)

        # Replace negative infinity values
        for col in train_data.columns:
            min_val = train_data[train_data[col] != -np.inf][col].min()
            train_data[col].replace(-np.inf, min_val, inplace=True)

        # Calculate the mean of each column
        column_means = train_data.mean()
        #Identify int col:
        int_columns = train_data.select_dtypes(include=['int64', 'int32', 'int8', 'Int64']).columns
        #round the mean for those columns:
        column_means[int_columns] = column_means[int_columns].round()
        # Fill NaN values in each column with the corresponding mean value
        train_data.fillna(column_means, inplace=True)

        #Restructure the dtypes for memory optimisation
        train_data = optimize_int_memory(train_data)
        #y is just the target column
        y = train_data['TARGET'].copy()
        y = 1 - y

        # Get the columns to be included in X
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]
        #Remove characters that model can't read
        X.columns = X.columns.str.replace('[^\w\s]','')
        
        # Scaling the data.
        scaler = StandardScaler()
        X_scaled_array = scaler.fit_transform(X)
        X = pd.DataFrame(X_scaled_array, columns=X.columns)

        #Clean unused data to free up ram
        del train_data, X_scaled_array
        gc.collect()
        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

    elif dataset_version == '4.0':
        # Same process as '3.0', but with additional PCA step.
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']
        train_data.drop(columns='level_0', inplace=True)

        #y is just the target column
        y = train_data['TARGET'].copy()
        y = 1 - y

        # Get the columns to be included in X.
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]

        # Replace positive infinity values
        for col in train_data.columns:
            max_val = train_data[train_data[col] != np.inf][col].max()
            train_data[col].replace(np.inf, max_val, inplace=True)

        # Replace negative infinity values
        for col in train_data.columns:
            min_val = train_data[train_data[col] != -np.inf][col].min()
            train_data[col].replace(-np.inf, min_val, inplace=True)

        # Calculate the mean of each column
        column_means = train_data.mean()
        #Identify int col:
        int_columns = train_data.select_dtypes(include=['int64', 'int32', 'int8', 'Int64']).columns
        #round the mean for those columns:
        column_means[int_columns] = column_means[int_columns].round()
        # Fill NaN values in each column with the corresponding mean value
        train_data.fillna(column_means, inplace=True)

        #Restructure the dtypes for memory optimisation
        train_data = optimize_int_memory(train_data)

        # Scaling the data.
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        # Apply PCA.
        pca = PCA(n_components=0.95)
        X = pca.fit_transform(X)

        # Convert back to DataFrame.
        X = pd.DataFrame(X, columns=["PC" + str(i) for i in range(1, X.shape[1] + 1)])

        del train_data
        gc.collect()
        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


    if dataset_version == '4.1':
        train_data = pd.read_feather("./train_data.feather")
        not_usable_col = ['TARGET', 'SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV', 'index', 'level_0']
        # Get the columns to be included in X.
        columns_X = [col for col in train_data.columns if col not in not_usable_col]
        X = train_data[columns_X]

        y = train_data.pop('TARGET')
        y = 1 - y
        X = train_data

        # Data splitting
        X, _, y, _ = train_test_split(X, y, stratify=y, test_size=0.9, random_state=42)
    
        # Replace infinity values more efficiently
        inf_mask = X == np.inf
        max_values = X[~inf_mask].max()
        X.where(~inf_mask, max_values, axis=1, inplace=True)

        minus_inf_mask = X == -np.inf
        min_values = X[~minus_inf_mask].min()
        X.where(~minus_inf_mask, min_values, axis=1,inplace=True)

        # Fill NaN values with column means
        int_columns = X.select_dtypes(include=['int64', 'int32', 'int8', 'Int64']).columns
        column_means = X.mean()
        column_means[int_columns] = column_means[int_columns].round()
        X.fillna(column_means, inplace=True)

        # Memory optimization
        X = optimize_int_memory(X)
        

        # Scaling
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        # PCA
        pca = PCA(n_components=0.95)
        X = pca.fit_transform(X)
        X = pd.DataFrame(X, columns=["PC" + str(i) for i in range(1, X.shape[1] + 1)])

        del train_data
        gc.collect()
        Xtrain, Xvalid, ytrain, yvalid =  train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


    print(X.shape)
    print(y.shape)
    ratio = y.sum()/len(y)*100
    print("Target 1-0 ratio: {:.2f}%".format(ratio))
    return Xtrain, ytrain, Xvalid, yvalid, dataset_version

In [ ]:
def save_5_rows(X, y):

    X_5 = X.head()
    y_5 = y.head()

    X_5.to_csv('./app/test/X_head', index = False)
    y_5.to_csv('./app/test/y_head',index = False)

    return 

**Let's create functions that will become our baseline for model iteration**

After we have our data we will put it through this function and it will get us:
- The best hyperparameters for those data and this model
- Save all information about the model run so we can compare with other results and pick the best model at the end 

In [ ]:
models = {
    'randomForestClassifier': lambda params: RandomForestClassifier(n_estimators=params['n_estimators'], max_depth=params['max_depth'], random_state=42),
    'lightGBM': lambda params: lgb.LGBMClassifier(num_leaves=params['num_leaves'], max_depth=params['max_depth'], n_estimators=params['n_estimators'], random_state=42),
    'logisticRegression': lambda params: LogisticRegression(C=params['C'], max_iter=1500, solver='sag'),
    'dummyClassifier': lambda params: DummyClassifier(strategy='constant', constant=0),
    'fullLGBM' : lambda params: lgb.LGBMClassifier(num_leaves=params['num_leaves'], max_depth=params['max_depth'], n_estimators=params['n_estimators'], random_state=42),
    #'SVC': lambda params: SVC(C=params['C'], gamma=params['gamma'], probability=True, random_state=42)
    # Add more models here as needed
}

In [ ]:
def train_and_evaluate_model(model_name, params, X, y, use_smote = False, threshold = 0.5):
    skf = StratifiedKFold(n_splits=5)

    accuracies = []
    adjusted_scores = []
    recalls = []
    f1_scores = []
    all_y_pred_prob = []  
    all_y_true = []
    fps=[]
    fns=[]


    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]


        if use_smote:
            #smote = SMOTE()
            smote = BorderlineSMOTE(kind='borderline-1')
            X_train, y_train = smote.fit_resample(X_train, y_train)
            #ratio = y_train.sum()/len(y_train)*100
            #print("Target 1-0 ratio: {:.2f}%".format(ratio))
            
        #Create the model with given param
        model = models[model_name](params)
        model.fit(X_train, y_train)

        #calculate the pred probabilities on the test set for ROC AUC later
        y_pred_ = model.predict_proba(X_val)

        # Adjust classification threshold
        y_pred = (y_pred_[:,0] < threshold).astype(int)

        #Calculate accuracy
        accuracy = accuracy_score(y_val, y_pred)
        accuracies.append(accuracy)

        #Calculate recall and f1_score
        recall = recall_score(y_val, y_pred)
        recalls.append(recall)

        f1 = f1_score(y_val, y_pred)
        f1_scores.append(f1)

        #accuracy is not everything, in our case errors on FP and FN are vastly different FN are lossing the bank much more money than FP
        tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
        if use_smote:
            adjusted_score =  fn + 10 * fp 
        else:
            adjusted_score =  fn + 90 *fp
        
        fps.append(fp)    
        fns.append(fn)    
        adjusted_scores.append(adjusted_score)

        all_y_pred_prob.extend(y_pred_[:,1])
        all_y_true.extend(y_val)

    mean_accuracy = np.mean(accuracies)
    mean_adjusted_score = np.mean(adjusted_scores)
    mean_recall = np.mean(recalls)
    mean_f1_score = np.mean(f1_scores)
    mean_fp = np.mean(fps)
    mean_fn = np.mean(fns)

    return model, mean_accuracy, all_y_true, all_y_pred_prob, mean_recall, mean_f1_score, mean_fp, mean_fn, adjusted_scores, mean_adjusted_score

In [ ]:
def objective(model_name, params, X, y, use_smote : bool, threshold = 0.5):
    """
    This function is what optuna will try to minimze during the best param search 
    """
    _, _, _, _, _, _, _, _, _, adjusted_score= train_and_evaluate_model(model_name, params,X,y,use_smote,threshold)
    

    return adjusted_score


In [ ]:
def run_optuna_experiment(model_name : Literal['lightGBM','randomForestClassifier','logisticRegression','dummyClassifier'],
                           X, y, max_trials:int, use_smote = False):
    
    #define the pruner
    pruner = MedianPruner(n_startup_trials=1, n_warmup_steps=5, interval_steps=1)
    # Create an Optuna study object
    study = optuna.create_study(direction='minimize', pruner=pruner)  # 'minimize' for custom "métier" score

    # Optimize the objective function (number of trials specified by max_trials)
    study.optimize(lambda trial: objective(model_name, 
                                       {'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                                        'max_depth': trial.suggest_int('max_depth', 3, 10)
                                       } if model_name == 'randomForestClassifier' else

                                       {'num_leaves': trial.suggest_int('num_leaves', 31, 150),
                                        'max_depth': trial.suggest_int('max_depth', -1, 20),
                                        'n_estimators': trial.suggest_int('n_estimators', 100, 600)
                                       } if model_name == 'lightGBM' else
                                       {
                                        'C': trial.suggest_float('C', 0.001, 100)
                                        } if model_name == 'logisticRegression' else
                                       {   
                                       }if model_name == 'dummyClassifier'else
                                        # Add more hyperparameters as needed
                                       {
                                        #Write Hparams for other models
                                       }, 
                                       X, y, use_smote, threshold = trial.suggest_float('threshold', 0.1, 0.6)), n_trials=max_trials)

    # Retrieve the best hyperparameters from the study
    best_params = study.best_params
    return best_params


In [ ]:
def run_best_model(model_name : Literal['lightGBM','randomForestClassifier','logisticRegression','dummyClassifier'],
                    best_params, X, y, dataset_version:str, use_smote = False, use_shap = False):
    
        # Start an MLflow run to track the training process
    with mlflow.start_run():
        # Log the dataset version
        mlflow.log_param('dataset_version', dataset_version)
        mlflow.log_param('SMOTE', use_smote)
        mlflow.log_param('model type', model_name)

        # Log the hyperparameters
        mlflow.log_params(best_params)
        threshold = best_params['threshold']
        # Train and evaluate the model using the best hyperparameters
        model, accuracy, all_y_true, y_pred_prob, recall, f1_score,fp, fn, adjusted_scores, adjusted_score = train_and_evaluate_model(model_name, best_params, X, y, use_smote,threshold)

        # Log the metrics
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('recall', recall)
        mlflow.log_metric('false_positive', fp)
        mlflow.log_metric('false_negative', fn)
        mlflow.log_metric('f1_score', f1_score)
        mlflow.log_metric('adjusted_score', adjusted_score)
        for i, score in enumerate(adjusted_scores, 1):
            mlflow.log_metric(f'adjusted_score_{i}', score)

        # Save the model as an artifact
        mlflow.sklearn.log_model(model, 'model')

        #region ROC AUC
        # Calculate ROC AUC
        roc_auc = roc_auc_score(all_y_true, y_pred_prob)
        mlflow.log_metric('roc_auc', roc_auc)
        # Plot ROC curve
        fpr, tpr, thresholds = roc_curve(all_y_true, y_pred_prob)
        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        
        # Save the plot as an image file
        roc_curve_img_path = './Exports/roc_curve.png'
        plt.savefig(roc_curve_img_path)

        # Log the ROC curve image as an artifact
        mlflow.log_artifact(roc_curve_img_path)

        # Close the plot to free up memory
        plt.close()
        # endregion 

        #region Shap
        if use_shap:            
            #local importance:
            index = 1

            if model_name == 'dummyClassifier':
                return  
            elif model_name == 'logisticRegression':
                explainer = shap.Explainer(model, X)
                shap_values = explainer(X)
                shap_html = shap.plots.force(shap_values[index], show=False)
                html_filepath = f"./Exports/shap_force_plot_{index}.html"
                shap.save_html(html_filepath, shap_html)

            else: #TreeBased model
                explainer = shap.TreeExplainer(model)
                shap_values = explainer.shap_values(X)
                shap_html = shap.force_plot(explainer.expected_value[1], shap_values[1][0,:], X.iloc[0,:], show=False)
                # Save the HTML content to a file
                html_filepath = f"./Exports/shap_force_plot_{index}.html"
                shap.save_html(html_filepath, shap_html)

            # Now for the full model feature importance
            shap.summary_plot(shap_values, X, show=False)
            plt.savefig('./Exports/shap_summary.png')
    
            # Log the SHAP force plot and summary plot as artifacts
            mlflow.log_artifact(html_filepath)
            mlflow.log_artifact('./Exports/shap_summary.png')
        #endregion
    return model

In [ ]:
def validate_model(Xvalid, yvalid, model,best_params):
    threshold = best_params['threshold']
    

    y_pred_prob = model.predict_proba(Xvalid)
    y_pred_prob_0 = y_pred_prob[:,0]
    y_pred_prob_1 = y_pred_prob[:,1]
    # Adjust classification threshold
    y_pred = (y_pred_prob_0 < threshold).astype(int)
    accuracy = accuracy_score(yvalid, y_pred)
    
    #Calculate recall and f1_score
    recall = recall_score(yvalid, y_pred)
    
    f1 = f1_score(yvalid, y_pred)

    roc_auc = roc_auc_score(yvalid, y_pred_prob_1)

    print('accuracy: ', accuracy)
    print('roc_auc: ', roc_auc)
    print('recall: ', recall)
    print('f1 score: ', f1)
    

In [ ]:
#To start the mlflow server to see the dashboard //// Be sure to change your directory to the app folder
#mlflow server --backend-store-uri sqlite:///mlflow.db

# Check if the experiment already exists
mlflow.set_tracking_uri('sqlite:///app/mlflow.db')

experiment_id = mlflow.get_experiment_by_name('credit_models')
if experiment_id is None:
    mlflow.create_experiment('credit_models', artifact_location='./app/mlruns/1')
    print('Experiment created')
else:
    print(f"Experiment 'credit_models' already exists with id {experiment_id.experiment_id}.")
    mlflow.set_experiment('credit_models')




In [ ]:
X, y, Xvalid, yvalid, current_version = change_dataset_version('2.0')

In [ ]:
#Use this to save some data for the model you put in production to test it 
save_5_rows(X,y)

In [ ]:
# best_optuna_params = run_optuna_experiment('logisticRegression',X,y,10,use_smote=True)

In [ ]:
best_optuna_params = {'C': 49.11321757734528, 'threshold': 0.40447759968727737}

In [ ]:
model = run_best_model('logisticRegression',best_optuna_params,X,y,current_version,use_smote=True,use_shap=True)

In [ ]:
#validate model
validate_model(Xvalid, yvalid, model, best_optuna_params)

------------------------------------------------
**Let's see how the predict part is going to look like**

----------------------------------------------------

In [ ]:
X1 = X.head(30)
y1 = y.head(30)

#set the current model ID and load the model f4f4d1af1ed34f8d9497b2ce38e9581e eacb0575c389406783c47125c5f3ce85
# Winner : 69c6dfea00f44d549419e7de4cf5262c
model_id = '69c6dfea00f44d549419e7de4cf5262c'
model_uri = f"./app/mlruns/1/{model_id}/artifacts/model"
model = mlflow.pyfunc.load_model(model_uri)
mlflow_client = mlflow.tracking.MlflowClient(tracking_uri='http://0.0.0.0:5000')
run = mlflow_client.get_run(run_id=model_id)
threshold = run.data.params['threshold']
threshold = float(threshold)

In [ ]:
threshold

In [42]:
X.tail()

,level_0,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
15168,130054.0,1,0,1,0,112500.0,469147.5,22698.0,351000.0,0.035792,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0
515,233135.0,1,0,0,2,112500.0,814041.0,23931.0,679500.0,0.018634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7031,268242.0,0,1,0,1,337500.0,1546020.0,45202.5,1350000.0,0.008019,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.0
13014,59973.0,0,1,0,0,351000.0,677664.0,43438.5,585000.0,0.018634,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24547,28201.0,1,0,0,0,45000.0,315000.0,15750.0,NaN,0.019101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
class WrappedModel:
    def __init__(self, mlflow_model):
        self.model = mlflow_model

    def predict(self, X):
        return self.model.predict(X)

    def predict_proba(self, X):
        if hasattr(self.model._model_impl, "predict_proba"):
            return self.model._model_impl.predict_proba(X)
        else:
            raise AttributeError("Underlying model does not have a 'predict_proba' method.")

wrapped_model = WrappedModel(model)
proba = wrapped_model.predict_proba(X1)
prediction = (proba[:,0] < threshold).astype(int)


api_answers = []
for i in range(0, len(prediction)):
    api_answer = [prediction[i], proba[i][0], proba[i][1]]
    api_answers.append(api_answer)
display(api_answers)

In [ ]:
model.predict